In [51]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from feature_engine.encoding import RareLabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from utils import ScalerDf, split_data
from pycaret.classification import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
salary_data = pd.read_csv('./../data/raw/ds_salaries.csv')

ValueError: Only callable can be used as callback

In [ ]:
quantiles = [0, 1/7, 2/7, 3/7, 4/7, 5/7, 6/7, 1]
bin_edges = [salary_data['salary'].quantile(q) for q in quantiles]

# Convert the continuous salary variable into 7 discrete bins based on quantiles
salary_labels = ['low', 'low-mid', 'mid', 'mid-high', 'high', 'very-high', 'Top']
salary_data['salary_range'] = pd.cut(salary_data['salary'], bins=bin_edges, labels=False, include_lowest=True)

In [38]:
X = salary_data.drop(columns=['salary', 'salary_in_usd', 'salary_range'])
y = salary_data['salary_range']
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y, test_size=0.2, val_size=0.25, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(2253, 9)
(751, 9)
(751, 9)


In [39]:
# identifico variables numericas y categoricas
categoric_columns = X.select_dtypes(exclude=['int', 'float']).columns.tolist()
numeric_columns = X.select_dtypes(include=['int', 'float']).columns.tolist()

In [40]:
preprocess_pipeline = [
    ('rare_label_encoder', RareLabelEncoder(variables=categoric_columns, n_categories=1, tol=0.1)),
    ('Ordinal_encoder',OrdinalEncoder(variables=categoric_columns, encoding_method='ordered')),
    ('Scaler', ScalerDf(method='standard'))
]
salary_preprocess = Pipeline(preprocess_pipeline)

In [43]:
y_train

2643    0
1655    2
661     1
1627    3
143     5
       ..
1301    1
1273    4
311     2
2358    6
1822    5
Name: salary_range, Length: 2253, dtype: Int64

In [44]:
salary_preprocess.fit(X_train, y_train)

TypeError: BaseMaskedArray.all() takes 1 positional argument but 2 were given

In [45]:
X_train['y'] = y_train
X_val['y'] = y_val

In [46]:
setup(  data= X_train, target = 'y',
            session_id = 0, 
            preprocess = False,
            test_data = X_val,
            index=False,
            verbose = False,custom_pipeline= salary_preprocess
                    )

TypeError: BlockManager.__init__() got an unexpected keyword argument 'parent'

In [47]:
best = compare_models(sort='AUC')

RuntimeError: This function/method requires the users to run setup() first.
More info: https://pycaret.gitbook.io/docs/get-started/quickstart

In [12]:
best_tuned = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2832,0.7041,0.2832,0.3043,0.2599,0.1631,0.1713
1,0.2832,0.6976,0.2832,0.3043,0.2740,0.1632,0.1676
2,0.3142,0.6964,0.3142,0.3144,0.2954,0.1995,0.2033
3,0.2978,0.6812,0.2978,0.3033,0.2844,0.1802,0.1827
4,0.2667,0.6471,0.2667,0.2684,0.2624,0.1441,0.1451
5,0.3156,0.7116,0.3156,0.2757,0.2876,0.2012,0.2046
6,0.3156,0.6863,0.3156,0.3507,0.2870,0.2010,0.2061
7,0.3111,0.7003,0.3111,0.3115,0.3033,0.1957,0.1985
8,0.3200,0.7209,0.3200,0.3554,0.3153,0.2065,0.2091


In [14]:
final_model = finalize_model(best)

In [16]:
pred = final_model.predict(X_train.drop(columns='y'))

In [17]:
from sklearn.metrics import classification_report, roc_auc_score

In [18]:
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

         0.0       0.63      0.74      0.68       319
         1.0       0.33      0.37      0.35       313
         2.0       0.35      0.28      0.31       321
         3.0       0.26      0.17      0.21       327
         4.0       0.35      0.11      0.17       324
         5.0       0.28      0.62      0.38       329
         6.0       0.31      0.20      0.24       320

    accuracy                           0.36      2253
   macro avg       0.36      0.36      0.33      2253
weighted avg       0.36      0.36      0.33      2253



In [24]:
predict_model(final_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.3409,0.7448,0.3409,0.3409,0.3228,0.2287,0.2339


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices